In [ ]:
import sys
sys.path.append('../')
import ta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#machine learning/statistical dependencies
import statsmodels.api as sm
from kneed import KneeLocator
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.manifold import TSNE
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima.model import ARIMA 
# from pykalman import KalmanFilter
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
import scipy.cluster.hierarchy as shc
import plotly.express as px
import gc
from collections import defaultdict
import seaborn as sns
import lightgbm as lgb

In [ ]:
from pair_trading_foundations.utils import GetSP500Data
import warnings
# The yfinance module has some deprecation warning
warnings.filterwarnings('ignore')

In [ ]:
import requests
from datetime import datetime
earliest_date = '2008-12-01'
todays_date = datetime.today().strftime('%Y-%m-%d')
session = requests.Session()
session.verify = False
print(f'Extracting date range: [ {earliest_date} ]-[ {todays_date} ]')
all_data = GetSP500Data(earliest_date,todays_date).get_all_sp_tickers().get_consolidated_data();

In [ ]:
final_data = all_data.copy()
final_data.head()

In [ ]:
# Examine Null Values in final_data. Examine Date, Close and daily_returns.
final_data.isna().sum()

In [ ]:
df = final_data.pivot(index='Date', columns='Ticker', values='Close') 
df.head()

In [ ]:
exp_data = df.copy()

# Null Values are Examined for Each Individual Stock on a Basis of Descriptive Statistics.

print('Null Values =',exp_data.isnull().values.any())

In [ ]:
 print('Data Shape Before Cleaning =', exp_data.shape)

missing_percentage = exp_data.isnull().mean().sort_values(ascending=False)
dropped_list = sorted(list(missing_percentage[missing_percentage > 0].index))
print(f"The following is a list of all stocks with ticker-level NA values: \n{dropped_list}")
exp_data.drop(labels=dropped_list, axis=1, inplace=True)

print('Data Shape After Cleaning =', exp_data.shape)

In [ ]:
exp_data = exp_data.fillna(exp_data.median()) ### This step to ensure we did not miss any odd days, and if we did, it is standard industry practice to take the median

exp_data['year']=exp_data.index.year

exp_data

In [ ]:
# prepare clustering data by splitting train and test to match trading strategy later

cluster_train_size = int(len(exp_data) * 0.7)

# Split the data into training, validation and test sets based on these lengths:
cluster_train = exp_data[0:cluster_train_size]
print(f"The length of the training set is: {len(cluster_train)}.")

In [ ]:
max(cluster_train.index)

In [ ]:
cluster_train

In [ ]:
# calculate average returns by year by stock
returns_year = cluster_train.groupby('year').mean()

# calculate return percent change, then aggregated returns and volatility by stock
returns = returns_year.pct_change().mean()

returns = pd.DataFrame(returns)
returns.columns = ['returns']
returns['volatility'] = returns_year.pct_change().dropna().std()

In [ ]:
returns

In [ ]:
ret_data = returns.copy()
ret_data.describe()

In [ ]:
# Standardize The Clustering Dataset Along a t-Distributed Curve
scale = StandardScaler().fit(ret_data)

# Transform the Original Clustering Dataset to These Standardized Values
scaled_data = pd.DataFrame(scale.fit_transform(ret_data),columns = ret_data.columns, index = ret_data.index) 
X = scaled_data
X.describe()

In [ ]:
n_clusters = 15
random_state = 15

# Algorithm:
# 1. Randomly pick k centroids from the examples ploted above as initial cluster centers (𝜇𝑗,𝑗∈1,...,𝑘)
# 2. Assign each example to the nearest centroid,  𝜇𝑗
# 3. Move the centroids to the center of the examples that were assigned to it
# 4. Repeat (2-3) until the cluster assignments do not change or a maximum number of iterations is reached.

def fit_kmeans(X, n_clusters, random_state):

    kmeans = KMeans(n_clusters=n_clusters,
                    init='k-means++',
                    n_init=15,
                    max_iter=300,
                    tol=1e-04)
    model = kmeans.fit(X)
    return model

model = fit_kmeans(X, n_clusters, random_state)
print(model)

## get cluster prediction and add to DataFrame
kX = X.copy()
kX['cluster'] = model.predict(X)
kX

In [ ]:
counts = clustered_series.value_counts()

cluster_vis_list = list(counts[(counts<20) & (counts>1)].index)[::-1]

clust1 = pd.DataFrame(clustered_series,columns=['label'])
grouped_df = clust1.groupby('label')

In [ ]:
final_data_train = final_data.copy()
final_data_train = final_data_train.loc[final_data_train.Date <= max(cluster_train.index)]

In [ ]:
pivot_data = final_data_train.pivot(index='Ticker', columns='Date', values='Adj Close')
pivot_data.head()

In [ ]:
# cluster evaluation and model selection
cluster_size_limit = 9999
counts = clustered_series.value_counts()
ticker_count = counts[(counts>1) & (counts<=cluster_size_limit)]
print ("Number of clusters: %d" % len(ticker_count))
print ("Number of Pairs: %d" % (ticker_count*(ticker_count-1)).sum())

In [ ]:
# Provide a functional definition for the cointegrated augmented dickey fuller test
def find_cadf_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = {}
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            ols = sm.OLS(S1,S2).fit()
            pred = ols.predict(S2)
            adf = ts.adfuller(S1-pred)
            score = adf[0]
            pvalue = adf[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.03:
                pairs[(keys[i], keys[j])] = adf
    return score_matrix, pvalue_matrix, pairs

In [ ]:
# Extract Cointegrated Pairs. This takes approximately 30-40 minutes. Again, this would take much longer if clustering analysis was not applied.
adf_dict = {}

for i, clust in enumerate(ticker_count.index):
    tickers = clustered_series[clustered_series == clust].index
    score_matrix, pvalue_matrix, adf_pairs = find_cadf_pairs(cluster_train[tickers])
    adf_dict[clust] = {}
    adf_dict[clust]['score_matrix'] = score_matrix
    adf_dict[clust]['pvalue_matrix'] = pvalue_matrix
    adf_dict[clust]['pairs'] = adf_pairs
    
adf_pairs = []   
for cluster in adf_dict.keys():
    adf_pairs.extend(adf_dict[cluster]['pairs'])
    
print ("Number of pairs:", len(adf_pairs))
print ("In those pairs, we found %d unique tickers." % len(np.unique(adf_pairs)))
print(adf_pairs)

In [ ]:
top500_pairs = list(map(list, adf_pairs))
top500_pairs = [lst for lst in top500_pairs if lst and not any(x is None for x in lst)]
top500_pairs

In [ ]:
# Modification of the ADF test from earlier to apply to the top pairs in each cluster. Pulling out the p-value only.
def find_cadf_pairs_validate(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = {}
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            ols = sm.OLS(S1,S2).fit()
            pred = ols.predict(S2)
            adf = ts.adfuller(S1-pred)
            score = adf[0]
            pvalue = adf[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs[(keys[i], keys[j])] = adf
    pvalue_result = min(pvalue_matrix[0])
    return pvalue_result

In [ ]:
# loop the best cluster list to filter exp_data df
# loop the filtered df through find_cadf_pairs_validate function to retrieve the pvalue_result
# append the pvalue_result to a new dataframe to rank the best pair based on p-value

def cluster_ranking(data):
    n = len(data)
    cluster_ranked = pd.DataFrame({'ticker1':[],'ticker2':[],'pvalue':[]})
    for i in range(n):
        pair_validate_df=exp_data.filter(items=data[i])
        p_value = find_cadf_pairs_validate(pair_validate_df)
        new_row = {'ticker1':top500_pairs[i][0],'ticker2':top500_pairs[i][1],'pvalue': p_value}
        cluster_ranked = cluster_ranked.append(new_row,ignore_index=True)
        new_row = {}
    cluster_ranked = cluster_ranked.sort_values(by='pvalue')
    return cluster_ranked


In [ ]:
# display the best pair from each cluster, sorted by the lowest p-value
top500_df = cluster_ranking(top500_pairs)

In [ ]:
top500_df = top500_df[0:500]

In [ ]:
top500_df = top500_df.reset_index()

In [ ]:
top500_df.to_csv('top500pairs.csv')